In [95]:
import numpy as np
import pandas as pd
import re

In [76]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN','londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
                    'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
                    'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                    'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )','12. Air France', '"Swiss Air"']})

In [48]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


Making correction in Flight Number. Filled in the NA values and changed the data type to int.
The flight number is meant to increase by 10 with each row so 10055 and 10075 need to be put in place.

In [77]:
for i,j in enumerate(df.FlightNumber):
    if np.isnan(j):
        try:
            df.FlightNumber[i] = df.FlightNumber[i-1]+10
        except:
            pass
    else:
        pass

df.FlightNumber = df.FlightNumber.astype(int)

C:\Users\yashg\AppData\Local\Temp/ipykernel_2460/3701541801.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.FlightNumber[i] = df.FlightNumber[i-1]+10


split From_to column and make the case consistent 

In [78]:
df[['From','To']] = df.From_To.str.split("_", expand= True)
df.drop('From_To', 1, inplace= True)
df.From = df.From.apply(lambda x:x.title())
df.To = df.To.apply(lambda x:x.title())

split delay column into multiple columns

In [82]:
df[['delay_1','delay_2', 'delay_3']] = pd.DataFrame(df.RecentDelays.tolist(), index= df.index)


clean airline data

In [93]:
def split_char(x):
    res = []
    splt = 0
    for i in range(len(x)-1):
        if x[i].islower() & x[i+1].isupper():
            res.append(x[:i+1])
            res.append(x[i+1:])
            splt = 1
    if splt == 1:
        res = " ".join(res)
    else:
        res = x
    return res

df.Airline = df.Airline.apply(lambda x:re.sub(r"[^a-zA-Z]", "", x))
df.Airline = df.Airline.apply(split_char)

In [94]:
df

,FlightNumber,RecentDelays,Airline,From,To,delay_1,delay_2,delay_3
0,10045,"[23, 47]",KLM,London,Paris,23.0,47.0,NaN
1,10055,[],Air France,Madrid,Milan,NaN,NaN,NaN
2,10065,"[24, 43, 87]",British Airways,London,Stockholm,24.0,43.0,87.0
3,10075,[13],Air France,Budapest,Paris,13.0,NaN,NaN
4,10085,"[67, 32]",Swiss Air,Brussels,London,67.0,32.0,NaN
